In [1]:
import os
import pandas as pd
import geopandas as gp
from geoalchemy2 import Geometry, WKTElement
from shapely.geometry.multipoint import MultiPoint
from shapely.geometry.multilinestring import MultiLineString
from shapely.geometry.multipolygon import MultiPolygon

from gispy.oss import lst_ff
from gispy.sql.con import alchemy_engine
from gispy.sql.db import create_db
from gispy.geofiles import get_epsg_shp

In [2]:
folder = '/home/useriam/gdata'

db = "workshop"

work_epsg = 3763

In [3]:
# List ESRI Shapefiles in folder

files = lst_ff(folder, file_format='.shp')

In [4]:
files

['/home/useriam/gdata/farmacias.shp',
 '/home/useriam/gdata/roads.shp',
 '/home/useriam/gdata/bgri.shp',
 '/home/useriam/gdata/lulc_coimbra.shp',
 '/home/useriam/gdata/freg_cmb.shp',
 '/home/useriam/gdata/hospitais.shp',
 '/home/useriam/gdata/ecopontos19.shp',
 '/home/useriam/gdata/e_desporto.shp']

In [5]:
# Create new database

create_db(db, overwrite=True, use_template=True)

'workshop'

In [22]:
bgri = gp.read_file('/home/useriam/gdata/bgri.shp')

cols = bgri.columns.values

ccols = {}
for c in cols:
    if c.isupper():
        ccols[c] = c.lower()

bgri.rename(columns=ccols, inplace=True)

In [23]:
bgri

,bgri11,lug11desig,idfreg,popres,wthgen_nd,geometry
0,06032600107,Marmeleira do Botão,060339,9,19.292038,"POLYGON ((-24855.631 71358.156, -24852.823 713..."
1,06030800213,Botão,060339,17,19.912100,"POLYGON ((-22803.793 71165.615, -22809.403 711..."
2,06032600406,Souselas,060339,28,12.827732,"POLYGON ((-25118.186 68641.256, -25114.898 686..."
3,06032600404,Souselas,060339,71,15.989368,"POLYGON ((-24589.200 68774.170, -24583.240 687..."
4,06032600215,Souselas,060339,35,15.855217,"POLYGON ((-24589.200 68774.170, -24598.490 687..."
...,...,...,...,...,...,...
2509,06031400308,Ardezubre,060337,45,19.090695,"POLYGON ((-36851.360 62217.737, -36846.360 621..."
2510,06032100101,Fonte de São Pedro,060337,43,16.580398,"POLYGON ((-35551.657 63741.539, -35558.547 637..."
2511,06031400113,Fonte de São Pedro,060337,9,17.283357,"POLYGON ((-35998.136 63241.993, -36003.660 632..."
2512,06031400205,Casais de Vera Cruz,060337,37,22.796472,"POLYGON ((-37439.334 65793.519, -37441.856 657..."


In [24]:
# Import data into PostgreSQL

# Get connector to database
conobj = alchemy_engine(db)

def sanitize_geom(geom, GeomClass, epsg):
    if geom.geom_type.startswith('Multi'):
        return WKTElement(geom.wkt, srid=epsg)
        
    else:
        return WKTElement(GeomClass([geom]).wkt, srid=epsg)

for f in files:
    # Check if we need to project data
    srs_code = get_epsg_shp(f)
    
    # Read data with GeoPandas
    gdf = gp.read_file(f)
    
    # Sanitize column names
    cols = gdf.columns.values

    ccols = {}
    for c in cols:
        if c.isupper():
            ccols[c] = c.lower()

    gdf.rename(columns=ccols, inplace=True)
    
    # Project if necessary
    if srs_code != work_epsg:
        print("We need to project {}. EPSG={}".format(f, str(srs_code)))
        gdf = gdf.to_crs('EPSG:{}'.format(str(work_epsg)))
    
    # Get Geometry Type
    gdf['geotype'] = gdf.geometry.geom_type
    
    geo_type = pd.DataFrame({
        'cgeom' : gdf.groupby(['geotype'])['geotype'].agg('count')
    }).reset_index().geotype.tolist()
    
    geo_type = geo_type[0] if len(geo_type) == 1 else [g for g in geo_type if g.startswith('Multi')][0]
    print(geo_type)
    
    # Add data to database
    if geo_type.startswith('Multi'):
        MultiClass = MultiPolygon if geo_type == 'MultiPolygon' else \
            MultiLineString if geo_type == 'MultiLineString' else MultiPoint
        
        gdf["geom"] = gdf.geometry.apply(lambda x: sanitize_geom(x, MultiClass, work_epsg))
    
    else:
        gdf["geom"] = gdf.geometry.apply(lambda x: WKTElement(x.wkt, srid=work_epsg))
    
    gdf.drop("geometry", axis=1, inplace=True)
    
    gdf.to_sql(
        os.path.splitext(os.path.basename(f))[0], conobj,
        if_exists='replace', index=False,
        dtype={"geom" : Geometry(geo_type, srid=work_epsg)}
    )

Point
MultiLineString
Polygon
MultiPolygon
Polygon
Point
We need to project /home/useriam/gdata/ecopontos19.shp. EPSG=4326
Point
Point
